# Huấn luyện Transformer trên Google Colab

Notebook này giúp bạn chạy training model Transformer chuyên nghiệp ngay trên Colab.

## 1. Cài đặt Môi trường
Kết nối Google Drive để truy cập code và lưu checkpoint.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Di chuyển vào thư mục dự án (Thay đổi đường dẫn nếu upload chỗ khác)

In [ ]:
import os
# Giả sử bạn upload folder Transformer vào thư mục gốc của Drive
# Nếu để chỗ khác, hãy sửa đường dẫn này
PROJECT_PATH = '/content/drive/MyDrive/Transformer' 

if os.path.exists(PROJECT_PATH):
    os.chdir(PROJECT_PATH)
    print(f"Đã chuyển vào: {os.getcwd()}")
else:
    print(f"Không tìm thấy thư mục {PROJECT_PATH}. Hãy kiểm tra lại Google Drive.")

## 2. Cài đặt Thư viện
Cài các thư viện còn thiếu trên Colab (như pyyaml, regex...).

In [ ]:
!pip install -r requirements.txt

## 3. Xử lý Dữ liệu Clean (Tùy chọn)
Nếu bạn có dữ liệu Raw và muốn chạy clean data trên Colab (đọc từ `data/raw` -> lưu vào `data/clean`).

In [ ]:
# Chạy script clean data
!python scripts/clean_data.py

## 4. Tùy chỉnh Cấu hình (Quan trọng)
Bạn có thể sửa trực tiếp các tham số ở đây (như `batch_size`, `epochs`) rồi chạy cell này.
Nó sẽ ghi đè vào file `configs/default.yaml` trước khi train.

**Lưu ý về Inference (Decoding):**
- **Greedy Search (`beam_size: 1`, `decoding_method: "greedy"`)**: Nhanh, nhẹ, chọn từ có xác suất cao nhất tại mỗi bước. Phù hợp debug.
- **Beam Search (`beam_size: 5`, `decoding_method: "beam"`)**: Chậm hơn nhưng chất lượng dịch tốt hơn, biết chọn phương án tối ưu toàn cục. Dùng khi Test lấy điểm BLEU.
  - Chú ý: Phải sửa cả `beam_size > 1` VÀ `decoding_method: "beam"` thì Beam Search mới được kích hoạt.

In [ ]:
%%writefile configs/default.yaml
model:
  d_model: 512
  n_layers: 6
  heads: 8
  dropout: 0.1
  vocab_size: 32768 
  max_len: 128 # Tăng lên tùy bài toán

training:
  batch_size: 64 # Tăng lên 64 hoặc 128 nếu dùng GPU A100
  lr: 0.0005 # An toàn
  epochs: 20
  warmup_ratio: 0.07 # 7% steps
  save_dir: "checkpoints"
  log_dir: "logs"
  seed: 42
  num_workers: 4 # Tăng lên 4 hoặc 8 trên Colab để load dữ liệu nhanh hơn

inference:
  # Cấu hình Decoding (Dịch)
  beam_size: 1             # 1 = Greedy (Mặc định). Tăng lên 5 để dùng Beam Search.
  decoding_method: "greedy" # "greedy" hoặc "beam".
  length_penalty: 0.6      # Phạt câu ngắn/dài khi dùng Beam Search.
  eval_batch_size: 64

data:
  min_len: 3
  max_ratio: 3.0
  bpe_dropout: 0.1

wandb:
  enabled: false
  project: "transformer-from-scratch"

## 5. Build Tokenizer (Bắt buộc)
Huấn luyện lại Tokenizer để đảm bảo tính tương thích.

In [ ]:
!python scripts/train_tokenizer.py --data_dir data/clean

## 6. Data Filtering & Caching (Optional)
Run the data filtering script to clean, filter (by length/ratio) and cache the data before training.
This step is optional but recommended to ensure data quality.

In [ ]:
# Run data filtering script
!python scripts/filter_data.py --force --export-text

## 7. Kiểm tra GPU
Xem Colab cấp GPU gì (A100, T4...).

In [ ]:
!nvidia-smi

## 8. Huấn luyện (Training)
Chạy script training. 
Lưu ý: Dataset sẽ được Pre-tokenize lần đầu (mất khoảng 1-2 phút).

In [ ]:
# import wandb
# wandb.login(key="PASTE_YOUR_API_KEY_HERE")

# Chạy script train (Lệnh python tìm file trong thư mục hiện tại)
!python scripts/train.py

## 9. Kiểm tra Model (Test)
Chạy thử model trên tập Validation và Test với checkpoint vừa train.

In [ ]:
# Thay tên checkpoint bạn muốn test
checkpoint_path = "checkpoints/checkpoint_epoch_20.pt"
if os.path.exists(checkpoint_path):
    !python scripts/test_model.py --checkpoint {checkpoint_path}
else:
    print(f"Chưa thấy file {checkpoint_path}. Hãy kiểm tra lại folder checkpoints.")

## 10. Đánh giá BLEU Score (Test Only)
Tính điểm BLEU chuẩn quốc tế (SacreBLEU) trên tập Test, sử dụng Beam Search (Beam Size = 5) để có kết quả tốt nhất.

In [ ]:
checkpoint_path = "checkpoints/checkpoint_epoch_20.pt"
if os.path.exists(checkpoint_path):
    if os.path.exists("data/clean/test.en"):
        print("\n--- Đánh giá trên tập Test (Greedy) ---")
        !python scripts/calculate_bleu.py \
            --checkpoint {checkpoint_path} \
            --split test \
            --decoding_method greedy \
            --beam_size 1

    if os.path.exists("data/clean/test.en"):
        print("\n--- Đánh giá trên tập Test (Beam Search Size 5) ---")
        !python scripts/calculate_bleu.py \
            --checkpoint {checkpoint_path} \
            --split test \
            --decoding_method beam \
            --beam_size 5
    else:
        print("Không tìm thấy tập test (data/clean/test.en). Hãy kiểm tra lại.")
else:
    print("Chưa có checkpoint.")

## 11. Tải Model về máy (Tùy chọn)
Nếu muốn tải file checkpoint về máy cá nhân.

In [ ]:
from google.colab import files
# Thay tên file checkpoint bạn muốn tải
ckpt_path = 'checkpoints/checkpoint_epoch_20.pt'
if os.path.exists(ckpt_path):
    files.download(ckpt_path)
else:
    print(f"Chưa thấy file {ckpt_path}")

# 12. Generate Submission for Contest
Chạy cell bên dưới để sinh file dự đoán predictions.txt. Nhớ kiểm tra đường dẫn checkpoint.

In [ ]:
# === TẠO FILE SUBMISSION CHO CONTEST ===
# Chạy script để sinh file dự đoán từ model đã train
# Bạn có thể thay đổi đường dẫn checkpoint nếu muốn
!python scripts/generate_submission.py \
    --test_src data/clean/test.en \
    --output predictions.txt \
    --config configs/default.yaml \
    --checkpoint_path checkpoints/checkpoint_epoch_10.pt